In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model._logistic import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors._classification import KNeighborsClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC
import numpy as np
from scipy import stats
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.float_format = "{:,.8f}".format

input_data = '/Users/kunsitu/Desktop/spotify/playlist_summary_external.txt'
df = pd.read_csv(input_data, sep = '	')
df_modified = pd.read_parquet('/Users/kunsitu/Downloads/playlist_summary_external_modified.parquet') # save from duckdb

In [3]:
df_modified.head()

,playlist_uri,owner,streams,stream30s,dau,wau,mau,mau_previous_month,mau_both_months,users,...,stream30s_scaled,monthly_stream30s_scaled,mau_both_months_scaled,dau_scaled,wau_scaled,skippers_scaled,stream30s_cohort,skippers_pct,weighted_streams_metric,weighted_au_metric
0,spotify:user:977723f001dc75663d2fcfe55b9b4b70:...,977723f001dc75663d2fcfe55b9b4b70,27,27,1,1,3,3,0,8,...,0.00,0.00,0.00,0.00,0.00,1.00,above 10,0.00,0.00,0.20
1,spotify:user:611d6958470da7b36b3a7b5700462f74:...,611d6958470da7b36b3a7b5700462f74,0,0,0,1,2,1,1,3,...,0.00,0.00,0.00,0.00,0.00,1.00,0,0.00,0.00,0.20
2,spotify:user:e2ff3a8d0187e4bd221e64fd924e7ea9:...,e2ff3a8d0187e4bd221e64fd924e7ea9,4,2,1,1,7,5,0,15,...,0.00,0.00,0.00,0.00,0.00,1.00,1-3,0.00,0.00,0.20
3,spotify:user:6719b0a5dc93e068a3c4f04081bcce6b:...,6719b0a5dc93e068a3c4f04081bcce6b,12,12,1,1,4,6,1,10,...,0.00,0.00,0.00,0.00,0.00,1.00,above 10,0.00,0.00,0.20
4,spotify:user:99d635f08ed668cdf7e36540fb653276:...,99d635f08ed668cdf7e36540fb653276,20,4,1,1,2,1,1,2,...,0.00,0.00,0.00,0.00,0.00,1.00,3-10,0.50,0.00,0.20


In [4]:
df_modified.describe()

,streams,stream30s,dau,wau,mau,mau_previous_month,mau_both_months,users,skippers,n_tracks,...,streams_scaled,stream30s_scaled,monthly_stream30s_scaled,mau_both_months_scaled,dau_scaled,wau_scaled,skippers_scaled,skippers_pct,weighted_streams_metric,weighted_au_metric
count,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,...,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00,403366.00
mean,71.01,42.83,4.42,21.78,66.14,58.19,12.94,149.31,2.83,201.48,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.05,0.00,0.20
std,6492.01,3772.41,358.86,1614.65,4732.58,3827.25,1240.91,9247.48,205.06,584.08,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.12,0.00,0.00
min,0.00,0.00,0.00,0.00,2.00,0.00,0.00,2.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.20
25%,0.00,0.00,0.00,1.00,2.00,1.00,1.00,2.00,0.00,38.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.20
50%,1.00,0.00,0.00,1.00,2.00,2.00,1.00,3.00,0.00,84.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.20
75%,8.00,5.00,1.00,2.00,4.00,3.00,2.00,7.00,0.00,192.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.20
max,2629715.00,1513237.00,152929.00,669966.00,1944150.00,1478684.00,578391.00,3455406.00,86162.00,79984.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,0.80


## classfication problem
- y = stream30s_cohort
- exclude some columns 
- outlier detection 
- train test split 
- desgin and run model
- show the accuarcy of the model

In [74]:
col_list = df.columns.to_list() + ['stream30s_cohort']
col_list = [i for i in col_list if i not in ['playlist_uri', 'owner', 'tokens','owner_country', 'stream30s']] # these columns are quite unique
data_main = df_modified[col_list]
data_main['stream30s_cohort'].value_counts()

stream30s_cohort
0           212652
above 10     99332
3-10         57056
1-3          34326
Name: count, dtype: int64

In [92]:
# detect outlier using z score
# abs above 3 filter it out
# numerical cols 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
num_cols = [data_main.columns[i] for i in range(len(data_main.columns)) if data_main.dtypes[i] != 'object']
obj_cols = [data_main.columns[i] for i in range(len(data_main.columns)) if data_main.dtypes[i] == 'object']
df_num = data_main[num_cols]
df_obj = data_main[obj_cols]

# sacle numerical data
scaler = MinMaxScaler()
df_num_scaled = scaler.fit_transform(df_num)  # Returns a NumPy array
df_num_scaled = pd.DataFrame(df_num_scaled, columns=df_num.columns)  #

# label encoding
le = LabelEncoder()
for i in obj_cols:
    df_obj[i] = le.fit_transform(df_obj[i])

# z score
df_num_zscore = stats.zscore(df_num_scaled)
df_num_zscore.columns = [i + '_zscore' for i in num_cols]
df_all_1 = pd.concat([df_num, df_obj, df_num_zscore], axis = 1)
df_no_outlier = df_all_1[(np.abs(df_num_zscore) < 3).all(axis=1)]
df_outlier = df_all_1[(np.abs(df_num_zscore) > 3).any(axis=1)]
df_no_outlier.drop([i for i in df_no_outlier.columns if 'zscore' in i], axis = 1, inplace = True)

In [77]:
# train test split
X = df_no_outlier.drop(['stream30s_cohort'], axis = 1)
y = df_no_outlier['stream30s_cohort']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# need to balance the data, SMOTE will reduce the chance of overfitting
smote_nc = SMOTENC(categorical_features=[15,16,17,18,19], random_state=0)
X_train_sm, y_train_sm = smote_nc.fit_resample(X_train, y_train)

In [78]:
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True, 
                     custom_metric=None,
                     classifiers=[RandomForestClassifier, LogisticRegression, KNeighborsClassifier, AdaBoostClassifier, MLPClassifier],
                     )
models,predictions = clf.fit(X_train_sm, X_test, y_train_sm, y_test)

100%|██████████| 5/5 [07:45<00:00, 93.14s/it] 


In [91]:
models.to_csv('/Users/kunsitu/Desktop/spotify/models.csv')

# randomfroest classifer test

In [80]:
# random forest is the best model, so try that
clf = RandomForestClassifier()
clf.fit(X_train_sm, y_train_sm)
y_pred = clf.predict(X_test)


In [87]:
# f1_score(y_test, y_pred, average = 'macro')
print(classification_report(y_test, y_pred, target_names=['0', '1-3', '3-10', 'above 10']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     41783
         1-3       0.60      0.69      0.64      6652
        3-10       0.71      0.74      0.72     11014
    above 10       0.87      0.80      0.83     18080

    accuracy                           0.89     77529
   macro avg       0.79      0.81      0.80     77529
weighted avg       0.89      0.89      0.89     77529



In [88]:
df_feature = pd.DataFrame([X_train_sm.columns, clf.feature_importances_]).T
df_feature.columns = ['feature', 'importance']
df_feature.sort_values(by = 'importance', ascending = False)

,feature,importance
0,streams,0.37
1,dau,0.22
12,monthly_stream30s,0.05
13,monthly_owner_stream30s,0.04
8,n_tracks,0.04
11,n_albums,0.03
10,n_artists,0.03
7,skippers,0.02
14,genre_1,0.02
2,wau,0.02


In [89]:
from xgboost import XGBClassifier
clf = XGBClassifier(n_estimators=100,
                     max_depth=5,
                     max_leaves=64,
                     eta=0.1,
                     reg_lambda=0,
                     tree_method='hist',
                     eval_metric='logloss',
                     use_label_encoder=False,
                     random_state=1000,
                     n_jobs=-1)

clf.fit(X_train_sm, y_train_sm)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     41783
           1       0.60      0.70      0.65      6652
           2       0.71      0.74      0.73     11014
           3       0.87      0.80      0.83     18080

    accuracy                           0.89     77529
   macro avg       0.80      0.81      0.80     77529
weighted avg       0.90      0.89      0.89     77529



In [90]:
pd.DataFrame([X_train_sm.columns, clf.feature_importances_]).T


,0,1
0,streams,0.22
1,dau,0.63
2,wau,0.00
3,mau,0.00
4,mau_previous_month,0.00
5,mau_both_months,0.01
6,users,0.00
7,skippers,0.04
8,n_tracks,0.00
9,n_local_tracks,0.00
